In [ ]:
gsutil -m rm gs://lpr-temp/* > log 2>&1 || true
tail log

In [ ]:
src=gs://lpr-cdip-tess
dst=gs://lpr-temp
out=gs://lpr-cdip-seg

In [ ]:
kubectl delete jobs --all 2>&1 | tail 
kubectl delete pods --all 2>&1 | tail
kubectl delete services shuffle || true

In [ ]:
cat > kubetpl.yaml <<'EOF'
image: gcr.io/research-191823/simple
memory: 8G
app: shuffle
subdomain: shuffle
EOF

In [ ]:
kubetpl service | kubectl apply -f -

In [ ]:
gsutil ls $src | fgrep .tgz | cat -n | 
while read index src; do
base=$(basename $src .tgz)
echo $index $src $base
kubetpl job -L jobgroup=mapper -M 8G -I gcr.io/research-191823/simple -n mapper$index -c "
python3 /files/hocr2markers $src /dev/stdout | gsutil cp - $dst/$base-seg.tgz
" | kubectl apply -f -
done

In [ ]:
sleep 60
kubectl logs job.batch/mapper1 | tail

In [ ]:
kubectl wait jobs -l jobgroup=mapper --for=condition=complete --timeout=-1s